In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from spare_parts import get_company_list
from tradingview_ta import TA_Handler, Interval, Exchange
import ta

c:\Users\hp\anaconda3\envs\data_science\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
# main class
class bollinger_band:
    def __init__(self, ticker):
        self.ticker = ticker
        self.MAX_PROFIT_LIMIT = 0.2
        self.MAX_LOSS_LIMIT = -0.1
        self.current_position = 0
        self.output = 0
        self.positions = []
        self.logs = []
        # special - signal
        self.call = "" 
        self.signal = 0 # this signal changes once stock reaches the upper - bollinger band
    # define function
    def download_data(self):
        data_raw = yf.download(self.ticker+".NS", period="2y", interval = "1d")
        # calculating bollinger band
        volatile = ta.volatility.BollingerBands(data_raw['Close']) # type: ignore
        data_raw['bollinger_hband'] = volatile.bollinger_hband()
        data_raw['bollinger_lband'] = volatile.bollinger_lband()
        data_raw['bollinger_mavg'] = volatile.bollinger_mavg()
        data_raw['bollinger_pband'] = volatile.bollinger_pband()
        # calculating obv
        volatile_obv = ta.volume.OnBalanceVolumeIndicator(data_raw['Close'], data_raw['Volume']) #type: ignore
        data_raw['obv'] = volatile_obv.on_balance_volume()
        # calculating rsi
        data_raw['rsi'] = ta.momentum.RSIIndicator(data_raw['Close']).rsi() #type: ignore
        data_raw.dropna(inplace = True)
        data_raw['diff'] = data_raw['obv'].diff(periods=10) # slope
        #print(data_raw.columns)
        self.data_list = data_raw.to_numpy()
        return data_raw
    
    def signal_check(self):
        self.download_data()
        main_list = self.data_list[-10:]
        for i in main_list:
            if(i[3] >= i[6]):
                self.signal = 1
        print(self.signal)
    # main function
    def run_test(self):
        self.signal_check()
        i = self.data_list[-1]
        

            # --
        try:
                current_node = i
                # buying section
                # checking the bollinger signal
                if(self.signal == 1):
                    # checking [RSI] and [OBV - diff] condition
                    if(current_node[12] > 0 and current_node[11] > 50):  # initally current_node[11] > 50
                        data = TA_Handler(
                            symbol = self.ticker,
                            exchange="NSE",
                            screener="india",
                            interval=Interval.INTERVAL_1_DAY
                        )
                        x = data.get_analysis().summary #type: ignore
                        print(x)
                        if(x['RECOMMENDATION'] == "STRONG_BUY"):
                            self.call = "buy"
                else:
                    print("condtion not passed")
        except:
            self.call = "sell"
            



In [2]:
get_company_list = get_company_list()

In [3]:
get_company_list

(('Reliance Industries Limited', 'RELIANCE'),
 ('Tata Consultancy Services Limited', 'TCS'),
 ('HDFC Bank Limited', 'HDFCBANK'),
 ('Infosys Limited', 'INFY'),
 ('ICICI Bank Limited', 'ICICIBANK'),
 ('Hindustan Unilever Limited', 'HINDUNILVR'),
 ('State Bank of India', 'SBIN'),
 ('Bajaj Finance Limited', 'BAJFINANCE'),
 ('Housing Development Finance Corporation Limited', 'HDFC'),
 ('Bharti Airtel Limited', 'BHARTIARTL'),
 ('Kotak Mahindra Bank Limited', 'KOTAKBANK'),
 ('Wipro Limited', 'WIPRO'),
 ('HCL Technologies Limited', 'HCLTECH'),
 ('ITC Limited', 'ITC'),
 ('Adani Green Energy Limited', 'ADANIGREEN'),
 ('Asian Paints Limited', 'ASIANPAINT'),
 ('Bajaj Finserv Limited', 'BAJAJFINSV'),
 ('Adani Transmission Limited', 'ADANITRANS'),
 ('Avenue Supermarts Limited', 'DMART'),
 ('Larsen & Toubro Limited', 'LT'),
 ('Adani Total Gas Limited', 'ATGL'),
 ('Axis Bank Limited', 'AXISBANK'),
 ('Maruti Suzuki India Limited', 'MARUTI'),
 ('Titan Company Limited', 'TITAN'),
 ('Adani Enterprises Lim

In [11]:
data_1 = [i[1] for i in get_company_list]

In [12]:
dict_1 = {}
for i in data_1[500:1000]:
    try:
        target = bollinger_band(i)
        target.run_test()
        dict_1[i] = target.call
    except:
        pass


[*********************100%***********************]  1 of 1 completed
0
condtion not passed
[*********************100%***********************]  1 of 1 completed
0
condtion not passed
[*********************100%***********************]  1 of 1 completed
1
{'RECOMMENDATION': 'STRONG_BUY', 'BUY': 16, 'SELL': 0, 'NEUTRAL': 10}
[*********************100%***********************]  1 of 1 completed
0
condtion not passed
[*********************100%***********************]  1 of 1 completed
1
{'RECOMMENDATION': 'BUY', 'BUY': 15, 'SELL': 2, 'NEUTRAL': 9}
[*********************100%***********************]  1 of 1 completed
1
{'RECOMMENDATION': 'STRONG_BUY', 'BUY': 17, 'SELL': 0, 'NEUTRAL': 9}
[*********************100%***********************]  1 of 1 completed
1
{'RECOMMENDATION': 'STRONG_BUY', 'BUY': 16, 'SELL': 1, 'NEUTRAL': 9}
[*********************100%***********************]  1 of 1 completed
1
{'RECOMMENDATION': 'BUY', 'BUY': 14, 'SELL': 3, 'NEUTRAL': 9}
[*********************100%**************

In [13]:
x = pd.DataFrame(dict_1.items(), columns=['Company', 'Call'])

In [15]:
x[x['Call'] == 'buy'].to_csv("buy.csv")
print("hello world")